In [1]:
from spacecraft.satellite import SatelliteImplementation
from setup.initial_settings import SimulationSetupReader
from setup.two_line_element import TwoLineElementReader
from visualziations.visualizations import plot_orbit, plot_lla, plot_position, plot_magnetic_field_eci, plot_magnetic_field_sbf, plot_sun_vector_eci, plot_sun_vector_sbf
from visualziations.visualizations import plot_angular_velocity, plot_euler_angles, plot_pointing_error, plot_torque, plot_angular_acceleration
from spacecraft.sensors import MagnetometerImplementation, SunsensorImplementation, SensorFusionImplementation
import core.transformations as tr
import core.utilities as ut
from pathlib import Path

import numpy as np

setup = SimulationSetupReader(Path('setup/initial_settings.json'))
tle = TwoLineElementReader(Path('setup/tle'))
magnetometer = MagnetometerImplementation(setup)
sunsensor = SunsensorImplementation(setup)
sensor_fusion = SensorFusionImplementation(setup, ['triad', 'quest', 'ekf'], tr.euler_xyz_to_quaternion(setup.euler_angles))

satellite = SatelliteImplementation(setup, tle, magnetometer, sunsensor, sensor_fusion)
quaternion_prev = satellite.quaternion.copy()
satellite.state_vector.next_row()
ut.basic_state_vector(satellite)

n_iter = setup.iterations_info["Stop"]
for x in range(setup.iterations_info["Start"], n_iter, setup.iterations_info["Step"]):
    satellite.manage_actuators_sensors_timing()
    satellite.update_iteration(x)
    satellite.apply_rotation()

    mag_field_sbf, mag_field_eci = satellite.magnetic_field
    sun_vector_sbf, sun_vector_eci = satellite.sun_vector

    if x % setup.iterations_info["LogInterval"] == 0:
        print(f"Iteration {x} of {n_iter}")

    satellite.fuse_sensors(
        [mag_field_sbf, sun_vector_sbf],
        [mag_field_eci, sun_vector_eci],
        quaternion_prev
    )
    
    satellite.manage_modes()
    satellite.apply_detumbling()
    satellite.apply_pointing()

    satellite.state_vector.next_row()
    ut.basic_state_vector(satellite)
    quaternion_prev = satellite.quaternion.copy() 

satellite.state_vector.to_csv('simulation_state_vector.csv')

print(satellite.state_vector.to_dataframe()) 
plot_orbit(satellite.state_vector.to_dataframe(), setup)
plot_position(satellite.state_vector.to_dataframe())
plot_lla(satellite.state_vector.to_dataframe())
plot_magnetic_field_eci(satellite.state_vector.to_dataframe())
plot_magnetic_field_sbf(satellite.state_vector.to_dataframe()) 
plot_angular_velocity(satellite.state_vector.to_dataframe())
plot_euler_angles(satellite.state_vector.to_dataframe())
plot_pointing_error(satellite.state_vector.to_dataframe())
plot_torque(satellite.state_vector.to_dataframe())
plot_angular_acceleration(satellite.state_vector.to_dataframe())
plot_sun_vector_eci(satellite.state_vector.to_dataframe())
plot_sun_vector_sbf(satellite.state_vector.to_dataframe())


Iteration 0 of 10000
 Standard B-dot gain 
 Standard B-dot 
Mode check: adapt_magnetic=False, adapt_angular=False, proportional=False, modified=False
Iteration 100 of 10000
Iteration 200 of 10000
Iteration 300 of 10000
Iteration 400 of 10000
Iteration 500 of 10000
Iteration 600 of 10000
Iteration 700 of 10000
Iteration 800 of 10000
Iteration 900 of 10000
Iteration 1000 of 10000
Iteration 1100 of 10000
Iteration 1200 of 10000
Iteration 1300 of 10000
Iteration 1400 of 10000
Iteration 1500 of 10000
Iteration 1600 of 10000
Iteration 1700 of 10000
Iteration 1800 of 10000
Iteration 1900 of 10000
Iteration 2000 of 10000
Iteration 2100 of 10000
Iteration 2200 of 10000
Iteration 2300 of 10000
Iteration 2400 of 10000
Iteration 2500 of 10000
Iteration 2600 of 10000
Iteration 2700 of 10000
Iteration 2800 of 10000
Detumbling stopped (|ω|=0.50 deg/s). Pointing started.


TypeError: MagnetorquerImplementation.current_to_angular_acceleration() missing 1 required positional argument: 'current_per_axis'

In [ ]:
# TODO
# complete b-cross
# add bang-bang mode for b-dot
# test all b-dot modes
# add on-off time for actuators and sensors 
# use better visualizations (interactive, zoom in/out   )
# some optimization
# verify the descriptions and documentation

# OPTIONAL
# add more sensors
# add more actuators
# tests for main parameters


Documentation:

1. Whats in this code

2. Short intro
    what is adcs
    simulating environment
    simulating sensors and actuartors
    basic control

3. Threoretical background
    simlating envireonment (tle, magnetic field, sun position etc)
    reference frames and transformations
    sensors (magnetometer, sun sensor, star tracker, gyroscope)


In [ ]:
import numpy as np
#                      x                 y                   z              x               y
array1 = np.array([-0.0114, -        -0.0114,          0.0114]) #         -0.0114, -        0.0114
array2 = np.array([ 1.92709051e-05, -3.48258106e-05,  1.31687787e-05]) # 1.92709051e-05, -3.48258106e-05

array3 = [
    (array1[1]*array2[2] - array1[2]*array2[1]),
    (array1[2]*array2[0] - array1[0]*array2[2]),
    (array1[0]*array2[1] - array1[1]*array2[0])
]

array3_np = np.cross(array1, array2)    

print(array3)
print(array3_np)
